# ATTN: This script should be run AFTER language detection is completed.

# Drop non-English job descriptions

### START HERE IF SOURCING FROM DF_JOBS_RAW_LANGUAGE_DETECTED
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [1]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [2]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


Using MPS


0it [00:00, ?it/s]

<Figure size 640x480 with 0 Axes>

In [3]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_raw_language_detected.pkl')


In [4]:
# len = 62577
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62577 entries, 0 to 62576
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Search Keyword     62577 non-null  object 
 1   Platform           62577 non-null  object 
 2   Job ID             62577 non-null  object 
 3   Job Title          62577 non-null  object 
 4   Company Name       62574 non-null  object 
 5   Location           62577 non-null  object 
 6   Job Description    62577 non-null  object 
 7   Rating             3975 non-null   float64
 8   Employment Type    61995 non-null  object 
 9   Company URL        59263 non-null  object 
 10  Job URL            62577 non-null  object 
 11  Job Age            62577 non-null  object 
 12  Job Age Number     62577 non-null  object 
 13  Collection Date    62577 non-null  object 
 14  Data Row           58599 non-null  float64
 15  Tracking ID        58599 non-null  object 
 16  Industry           591

In [5]:
# nl = 44863, en = 17591, ['en', 'nl'] = 8, ['nl', 'en'] = 9
df_jobs['Language'].value_counts()


nl              44863
en              17591
de                 53
fr                 36
['nl', 'en']        9
['en', 'nl']        8
pl                  5
id                  4
da                  4
tr                  1
['nl', 'af']        1
st                  1
af                  1
Name: Language, dtype: int64

In [6]:
%%time
# The majority of ['en', 'nl'] labeled job descriptions contain mostly English
df_jobs['Language'] = df_jobs['Language'].progress_apply(
    lambda lang: 'en' if lang == "['en', 'nl']" else lang
)


progress-bar:   0%|          | 0/62577 [00:00<?, ?it/s]

CPU times: user 99.7 ms, sys: 7.67 ms, total: 107 ms
Wall time: 402 ms


In [7]:
# Drop non-English ads
df_jobs = df_jobs.drop(
    df_jobs[
        df_jobs['Language'] != 'en'
    ].index, 
        axis='index', 
)


In [8]:
# 17599
len(df_jobs)


17599

In [9]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_raw_english_only.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_raw_english_only.csv', index=False)


# Fix abbreviations in job descriptions

### START HERE IF SOURCING FROM DF_JOBS_RAW_ENGLISH_ONLY
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [10]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [11]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [12]:
# Function to make a list of punctuations that determine sentence boundry, i.e., split characters
def make_custom_punct_chars(main_punct_chars=None, repeated_punct_chars=None):
    if main_punct_chars is None:
        main_punct_chars = [':', '|']
    if repeated_punct_chars is None:
        repeated_punct_chars = ['\n', ',']
    custom_punct_chars = []
    temp_multi = []
    temp_spaced = []

    for punct_char in main_punct_chars:
        custom_punct_chars+= f'{punct_char}', f'{punct_char} '

    for idx in range(4):
        for punct_char in repeated_punct_chars:
            temp_multi.append(f'{punct_char}'*int(idx+1))
            temp_spaced.append(f'{punct_char} '*int(idx+1))

    for multi, spaced in zip(temp_multi, temp_spaced):
        custom_punct_chars+= multi, spaced

    custom_punct_chars.remove(',')
    custom_punct_chars.remove(', ')

    return custom_punct_chars


In [13]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_raw_english_only.pkl')


In [14]:
# len = 17599
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 17599 entries, 0 to 62535
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Search Keyword     17599 non-null  object 
 1   Platform           17599 non-null  object 
 2   Job ID             17599 non-null  object 
 3   Job Title          17599 non-null  object 
 4   Company Name       17597 non-null  object 
 5   Location           17599 non-null  object 
 6   Job Description    17599 non-null  object 
 7   Rating             3780 non-null   float64
 8   Employment Type    17017 non-null  object 
 9   Company URL        15959 non-null  object 
 10  Job URL            17599 non-null  object 
 11  Job Age            17599 non-null  object 
 12  Job Age Number     17599 non-null  object 
 13  Collection Date    17599 non-null  object 
 14  Data Row           13816 non-null  float64
 15  Tracking ID        13816 non-null  object 
 16  Industry           144

In [15]:
df_jobs['Job Description'] = df_jobs['Job Description'].astype(str)


In [16]:
df_jobs['Job Description'] = df_jobs['Job Description'].apply(lambda x: x.strip())


In [17]:
df_jobs['Job Description'] = df_jobs['Job Description'].apply(lambda x: unicodedata.normalize('NFKD', x.encode('ascii', 'ignore').decode('utf-8', 'ignore')))


In [18]:
df_jobs['Job Description'].head()


0    About Our Client\nThe Global KYC organisation ...
2    Your role:\n\nWere in business to save our hom...
3    During the past four years Colourful Rebel has...
5    Job Description\nWe are currently recruiting a...
7    KARL LAGERFELD COMPANY PROFILE\n\nThe house of...
Name: Job Description, dtype: object

In [19]:
# count=17599, unique=17594, top=, freq=1
df_jobs['Job Description'].describe()


count                                                 17599
unique                                                17594
top       At SoftwareONE we are customerfocused Delighti...
freq                                                      3
Name: Job Description, dtype: object

In [20]:
df_jobs = df_jobs.drop_duplicates(subset=['Job Description'], keep='first')


In [21]:
# len=17594
len(df_jobs)


17594

In [22]:
df_jobs['Job Description'].head()


0    About Our Client\nThe Global KYC organisation ...
2    Your role:\n\nWere in business to save our hom...
3    During the past four years Colourful Rebel has...
5    Job Description\nWe are currently recruiting a...
7    KARL LAGERFELD COMPANY PROFILE\n\nThe house of...
Name: Job Description, dtype: object

In [23]:
# count=17594, unique=17594, top=, freq=1
df_jobs['Job Description'].describe()


count                                                 17594
unique                                                17594
top       About Our Client\nThe Global KYC organisation ...
freq                                                      1
Name: Job Description, dtype: object

In [24]:
custom_punct_chars = make_custom_punct_chars()
print(custom_punct_chars)


[':', ': ', '|', '| ', '\n', '\n ', '\n\n', '\n \n ', ',,', ', , ', '\n\n\n', '\n \n \n ', ',,,', ', , , ', '\n\n\n\n', '\n \n \n \n ', ',,,,', ', , , , ']


In [25]:
# Drop na, empty strings, punctuations, and keep only sentences of length more than 0
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_chars = pickle.load(f)

df_jobs = df_jobs.dropna(subset=['Job Description'])
df_jobs = df_jobs.loc[
    (df_jobs['Job Description'].isnull() == False)
    & (df_jobs['Job Description'].isna() == False)
    & (df_jobs['Job Description'].notnull() == True)
    & (df_jobs['Job Description'].notna() == True)
    & (df_jobs['Job Description'].astype(bool) == True)
    & (df_jobs['Job Description'].apply(len) > 0)
    & (pd.isna(df_jobs['Job Description']) == False)
    & (df_jobs['Job Description'].astype(str).isin(custom_punct_chars) == False)
    & (df_jobs['Job Description'].astype(str).isin(non_whitespace_nan_list) == False)
    & (df_jobs['Job Description'].astype(str).isin(list(string.punctuation)) == False)
    & (df_jobs['Job Description'].astype(str).isin(list(string.whitespace)) == False)
]
df_jobs = df_jobs.dropna(subset=['Job Description'])


In [26]:
df_jobs['Job Description'].head()


0    About Our Client\nThe Global KYC organisation ...
2    Your role:\n\nWere in business to save our hom...
3    During the past four years Colourful Rebel has...
5    Job Description\nWe are currently recruiting a...
7    KARL LAGERFELD COMPANY PROFILE\n\nThe house of...
Name: Job Description, dtype: object

In [27]:
df_jobs['Job Description'].describe()


count                                                 17593
unique                                                17593
top       About Our Client\nThe Global KYC organisation ...
freq                                                      1
Name: Job Description, dtype: object

In [28]:
# Remove duplicate sentences from the same job ad
df_jobs = df_jobs.drop_duplicates(subset=['Job ID', 'Job Description'])


In [29]:
# Check row are string or invalid
for idx, row in df_jobs['Job Description'].items():
    if (not isinstance(row, str)) or (row in nan_list) or (row in custom_punct_chars) or (row in list(string.punctuation)) or (pd.isna(row) == True):
        print(idx, row)

In [30]:
# len=17593
len(df_jobs)


17593

In [31]:
df_jobs['Job Description'].head()


0    About Our Client\nThe Global KYC organisation ...
2    Your role:\n\nWere in business to save our hom...
3    During the past four years Colourful Rebel has...
5    Job Description\nWe are currently recruiting a...
7    KARL LAGERFELD COMPANY PROFILE\n\nThe house of...
Name: Job Description, dtype: object

In [32]:
# count=17593, unique=17593
df_jobs['Job Description'].describe()


count                                                 17593
unique                                                17593
top       About Our Client\nThe Global KYC organisation ...
freq                                                      1
Name: Job Description, dtype: object

In [33]:
df_jobs['Job Description'] = df_jobs.loc[df_jobs['Job Description'].notnull(), 'Job Description'].progress_apply(
    lambda job_description: ' '.join(job_description.split('/')) if '/' in job_description else job_description
)


progress-bar:   0%|          | 0/17593 [00:00<?, ?it/s]

In [34]:
abb_dict = {
    r'incl\.': 'including', 
    r'e\.g\.': 'for example', 
    r'e\.g': 'for example', 
    r'etc\.': 'et cetera', 
    r'\"': '',
}


In [35]:
%%time
df_jobs['Job Description'] = df_jobs['Job Description'].replace(abb_dict, regex=True)


CPU times: user 322 ms, sys: 4.21 ms, total: 326 ms
Wall time: 357 ms


In [36]:
df_jobs['Job Description'].head()


0    About Our Client\nThe Global KYC organisation ...
2    Your role:\n\nWere in business to save our hom...
3    During the past four years Colourful Rebel has...
5    Job Description\nWe are currently recruiting a...
7    KARL LAGERFELD COMPANY PROFILE\n\nThe house of...
Name: Job Description, dtype: object

In [37]:
# count= 17593, unique= 17593
df_jobs['Job Description'].describe()


count                                                 17593
unique                                                17593
top       About Our Client\nThe Global KYC organisation ...
freq                                                      1
Name: Job Description, dtype: object

In [38]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_raw_fixed_abbreviations.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_raw_fixed_abbreviations.csv', index=False)


# Add English and Dutch language requirement columns

### START HERE IF SOURCING FROM DF_JOBS_RAW_FIXED_ABBREVIATIONS
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [39]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [40]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [41]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_raw_fixed_abbreviations.pkl')


In [42]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 17593 entries, 0 to 62535
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Search Keyword     17593 non-null  object 
 1   Platform           17593 non-null  object 
 2   Job ID             17593 non-null  object 
 3   Job Title          17593 non-null  object 
 4   Company Name       17591 non-null  object 
 5   Location           17593 non-null  object 
 6   Job Description    17593 non-null  object 
 7   Rating             3780 non-null   float64
 8   Employment Type    17011 non-null  object 
 9   Company URL        15954 non-null  object 
 10  Job URL            17593 non-null  object 
 11  Job Age            17593 non-null  object 
 12  Job Age Number     17593 non-null  object 
 13  Collection Date    17593 non-null  object 
 14  Data Row           13810 non-null  float64
 15  Tracking ID        13810 non-null  object 
 16  Industry           143

In [43]:
%%time
# Add language requirement column
# Use regex to find language requirement
dutch_requirement_pattern = r'[Dd]utch [Pp]referred | [Dd]utch [Re]quired | [Dd]utch [Ll]anguage |[Pp]roficient in [Dd]utch |[Ss]peak [Dd]utch | [Kk]now [Dd]utch | [Ff]luent in [Dd]utch | [Dd]utch [Nn]ative | * [Dd]utch [Ll]evel'
english_requirement_pattern = r'[Ee]nglish [Pp]referred | [Ee]nglish [Re]quired | [Ee]nglish [Ll]anguage |[Pp]roficient in [Ee]nglish |[Ss]peak [Ee]nglish | [Kk]now [Ee]nglish | [Ff]luent in [Ee]nglish | [Ee]nglish [Nn]ative | * [Ee]nglish [Ll]evel'

lang_requirements = {
    'Dutch Requirement in Job Ad': dutch_requirement_pattern, 'English Requirement in Job Ad': english_requirement_pattern
}

for lang_req, lang_req_pattern in lang_requirements.items():

    if lang_req in df_jobs.columns:
        df_jobs = df_jobs.drop(columns=[lang_req])
    df_jobs[lang_req] = np.where(
        df_jobs['Job Description'].str.contains(lang_req_pattern),
        'Yes',
        'No',
    )
    df_jobs[lang_req] = df_jobs[lang_req].astype('category').cat.reorder_categories(['No', 'Yes'], ordered=True)
    df_jobs[lang_req] = pd.Categorical(df_jobs[lang_req], categories=['No', 'Yes'], ordered=True)

assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_raw_language_requirement.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_raw_english_requirement.csv', index=False)


CPU times: user 6.7 s, sys: 196 ms, total: 6.89 s
Wall time: 8.87 s


In [44]:
# Yes = 252 job ads
df_jobs['Dutch Requirement in Job Ad'].value_counts()

No     17341
Yes      252
Name: Dutch Requirement in Job Ad, dtype: int64

In [45]:
# Yes = 483 job ads
df_jobs['English Requirement in Job Ad'].value_counts()


No     17110
Yes      483
Name: English Requirement in Job Ad, dtype: int64

In [46]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_raw_language_requirement.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_raw_language_requirement.csv', index=False)


# Add data from Sectors dataframe (see CBS directory under scrapped_data directory)


### START HERE IF SOURCING FROM DF_JOBS_RAW_LANGUAGE_REQUIREMENT
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [47]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [48]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [49]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_raw_language_requirement.pkl')


In [50]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 17593 entries, 0 to 62535
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   Search Keyword                 17593 non-null  object  
 1   Platform                       17593 non-null  object  
 2   Job ID                         17593 non-null  object  
 3   Job Title                      17593 non-null  object  
 4   Company Name                   17591 non-null  object  
 5   Location                       17593 non-null  object  
 6   Job Description                17593 non-null  object  
 7   Rating                         3780 non-null   float64 
 8   Employment Type                17011 non-null  object  
 9   Company URL                    15954 non-null  object  
 10  Job URL                        17593 non-null  object  
 11  Job Age                        17593 non-null  object  
 12  Job Age Number                 1

In [51]:
try:
    df_sectors_all = pd.read_pickle(f'{scraped_data}CBS/Data/Sectors Output from script.pkl')
except FileNotFoundError:
    %run f'{scraped_data}CBS/CBS.ipynb' import df_sectors_all # type:ignore # isort:skip # fmt:skip # noqa # nopep8

In [52]:
df_sectors_all.columns


MultiIndex([('SBI Sector Titles', 'Industry class / branch (SIC2008)',                   'Code'),
            ('SBI Sector Titles', 'Industry class / branch (SIC2008)',            'Sector Name'),
            ('SBI Sector Titles', 'Industry class / branch (SIC2008)',               'Keywords'),
            ('SBI Sector Titles', 'Industry class / branch (SIC2008)',         'Keywords Count'),
            (           'Gender',                            'Female',                      'n'),
            (           'Gender',                            'Female',           '% per Sector'),
            (           'Gender',                            'Female',  '% per Social Category'),
            (           'Gender',                            'Female',        '% per Workforce'),
            (           'Gender',                              'Male',                      'n'),
            (           'Gender',                              'Male',           '% per Sector'),
            (       

In [53]:
df_sectors_all.columns = [
    '_'.join(col) 
    if 'SBI Sector Titles' not in col 
    and 'Total Workforce' not in col 
    else col[-1] 
    for col in df_sectors_all.columns
]


In [54]:
df_sectors_all = df_sectors_all.rename(
    columns={
        'Keywords': 'Search Keyword', 
        'Code': 'Sector Code', 
        'Sector Name': 'Sector', 
        'Gender_Sectoral Gender Segregation_Dominant Category': 'Gender', 
        'Age_Sectoral Age Segregation_Dominant Category': 'Age', 
        'n': 'Sector_n', 
    },
)
df_sectors_all = df_sectors_all.rename(columns={element: re.sub(r' \(\W*45 years\)', '', element) for element in df_sectors_all.columns.tolist()})


In [55]:
df_sectors_all.columns


Index(['Sector Code', 'Sector', 'Search Keyword', 'Keywords Count', 'Gender_Female_n', 'Gender_Female_% per Sector', 'Gender_Female_% per Social Category', 'Gender_Female_% per Workforce', 'Gender_Male_n', 'Gender_Male_% per Sector', 'Gender_Male_% per Social Category', 'Gender_Male_% per Workforce', 'Gender', 'Age_Older_n', 'Age_Older_% per Sector', 'Age_Older_% per Social Category', 'Age_Older_% per Workforce', 'Age_Younger_n', 'Age_Younger_% per Sector', 'Age_Younger_% per Social Category', 'Age_Younger_% per Workforce', 'Age', 'Sector_n', '% Sector per Workforce'], dtype='object')

In [56]:
df_sectors_all = df_sectors_all.explode(
    'Search Keyword', ignore_index=True
)


In [57]:
df_sectors_all.columns


Index(['Sector Code', 'Sector', 'Search Keyword', 'Keywords Count', 'Gender_Female_n', 'Gender_Female_% per Sector', 'Gender_Female_% per Social Category', 'Gender_Female_% per Workforce', 'Gender_Male_n', 'Gender_Male_% per Sector', 'Gender_Male_% per Social Category', 'Gender_Male_% per Workforce', 'Gender', 'Age_Older_n', 'Age_Older_% per Sector', 'Age_Older_% per Social Category', 'Age_Older_% per Workforce', 'Age_Younger_n', 'Age_Younger_% per Sector', 'Age_Younger_% per Social Category', 'Age_Younger_% per Workforce', 'Age', 'Sector_n', '% Sector per Workforce'], dtype='object')

In [58]:
# len 101
len(df_sectors_all)


101

#### Before adding sector data, make sure keywords are correct as to not have any missing sector data when merging

In [59]:
# This is a manually collected dictionary of incorrect/faulty keywords in scraped site data
with open(f'{scraped_data}CBS/Data/keyword_trans_dict.txt') as f:
    keyword_trans_dict = json.load(f)

def fix_keywords(df_temp):

    if len(df_temp) > 0 and isinstance(df_temp, pd.DataFrame):
        for key, value in keyword_trans_dict.items():
            df_temp.loc[
                df_temp[df_temp['Search Keyword'].notnull()]['Search Keyword'].astype(str).progress_apply(
                lambda x: x.lower().strip()
                ) == str(key).lower().strip(), 'Search Keyword'
            ] = str(value).lower().strip()

        unfixed = df_temp.loc[
            df_temp[df_temp['Search Keyword'].notnull()]['Search Keyword'].astype(str).progress_apply(lambda x: x.lower().strip()).isin([x.lower().strip() for x in list(keyword_trans_dict.keys())])
        ]

        if len(unfixed) != 0:
            for key, value in keyword_trans_dict.items():
                for idx, row in df_temp.iterrows():
                    if row['Search Keyword'].astype(str).lower().strip() == str(key).lower().strip():
                        df_temp.loc[idx, 'Search Keyword'] = str(value).lower().strip()

        unfixed = df_temp.loc[
                df_temp[df_temp['Search Keyword'].notnull()]['Search Keyword'].astype(str).progress_apply(lambda x: x.lower().strip()).isin([x.lower().strip() for x in list(keyword_trans_dict.keys())])
            ]
        if len(unfixed) != 0:
            print('Some keywords were not fixed. Please check file unfixed_keywords.txt in data directory.')
            with open(f'{data_dir}unfixed_keywords.txt', 'w') as f:
                json.dump(unfixed, f)

    return df_temp

# Fix keywords
if len(df_jobs['Search Keyword'].loc[df_jobs['Search Keyword'].isin(list(keyword_trans_dict.keys()))]) != 0:
    print('Some search keywords did not match a sector. Fixing')
#     print(set(df_jobs['Search Keyword'].loc[df_jobs['Sector'].isna()].to_list()))
    print(len(df_jobs['Search Keyword'].loc[df_jobs['Search Keyword'].isin(list(keyword_trans_dict.keys()))]))
    df_jobs = fix_keywords(df_jobs)
#     print(set(df_jobs['Search Keyword'].loc[df_jobs['Sector'].isna()].to_list()))
    print(len(df_jobs['Search Keyword'].loc[df_jobs['Search Keyword'].isin(list(keyword_trans_dict.keys()))]))


In [60]:
df_jobs = df_jobs.merge(df_sectors_all, on='Search Keyword', how='left')


In [61]:
# 17593
len(df_jobs)


17593

In [62]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 17593 entries, 0 to 17592
Data columns (total 45 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Search Keyword                       17593 non-null  object  
 1   Platform                             17593 non-null  object  
 2   Job ID                               17593 non-null  object  
 3   Job Title                            17593 non-null  object  
 4   Company Name                         17591 non-null  object  
 5   Location                             17593 non-null  object  
 6   Job Description                      17593 non-null  object  
 7   Rating                               3780 non-null   float64 
 8   Employment Type                      17011 non-null  object  
 9   Company URL                          15954 non-null  object  
 10  Job URL                              17593 non-null  object  
 11  Job Age        

#### Check if there is any missing sector data in the merged dataframe

In [63]:
df_jobs['Sector'].isna().sum()


0

In [64]:
if df_jobs['Sector'].isna().sum() != 0:
    print('Some search keywords did not match a sector. Fixing')
    print(set(df_jobs['Search Keyword'].loc[df_jobs['Sector'].isna()].to_list()))
    print(len(df_jobs['Search Keyword'].loc[df_jobs['Search Keyword'].isin(list(keyword_trans_dict.keys()))]))
    df_jobs = fix_keywords(df_jobs)
    print(set(df_jobs['Search Keyword'].loc[df_jobs['Sector'].isna()].to_list()))
    print(len(df_jobs['Search Keyword'].loc[df_jobs['Search Keyword'].isin(list(keyword_trans_dict.keys()))]))


In [65]:
# Female=3955, Male=6974, Mixed Gender=6664
df_jobs['Gender'].value_counts()


Male            6974
Mixed Gender    6664
Female          3955
Name: Gender, dtype: int64

In [66]:
# Older=3778, Younger=2351, Mixed Age=11464
df_jobs['Age'].value_counts()


Mixed Age    11464
Older         3778
Younger       2351
Name: Age, dtype: int64

In [67]:
if df_jobs['Sector'].isna().sum() == 0:
    assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
    df_jobs.to_pickle(f'{df_save_dir}df_jobs_including_sector_data.pkl')
    df_jobs.to_csv(f'{df_save_dir}df_jobs_including_sector_data.csv', index=False)

else:
    print(f"MISSING SECTOR DATA: COUNT {df_jobs['Sector'].isna().sum()}")


# Add categorical data


### START HERE IF SOURCING FROM DF_JOBS_INCLUDING_SECTOR_DATA
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [68]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [69]:
from setup_module.imports import * # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [70]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_including_sector_data.pkl')


In [71]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 17593 entries, 0 to 17592
Data columns (total 45 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Search Keyword                       17593 non-null  object  
 1   Platform                             17593 non-null  object  
 2   Job ID                               17593 non-null  object  
 3   Job Title                            17593 non-null  object  
 4   Company Name                         17591 non-null  object  
 5   Location                             17593 non-null  object  
 6   Job Description                      17593 non-null  object  
 7   Rating                               3780 non-null   float64 
 8   Employment Type                      17011 non-null  object  
 9   Company URL                          15954 non-null  object  
 10  Job URL                              17593 non-null  object  
 11  Job Age        

In [72]:
df_jobs = df_jobs.join(pd.get_dummies(df_jobs[['Gender', 'Age']], dtype='int64'))
df_jobs = df_jobs.rename({'Gender_Mixed Gender': 'Gender_Mixed', 'Age_Mixed Age': 'Age_Mixed'}, axis='columns')


In [73]:
df_jobs.columns


Index(['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Rating', 'Employment Type', 'Company URL', 'Job URL', 'Job Age', 'Job Age Number', 'Collection Date', 'Data Row', 'Tracking ID', 'Industry', 'Job Date', 'Type of ownership', 'Language', 'Dutch Requirement in Job Ad', 'English Requirement in Job Ad', 'Sector Code', 'Sector', 'Keywords Count', 'Gender_Female_n', 'Gender_Female_% per Sector', 'Gender_Female_% per Social Category', 'Gender_Female_% per Workforce', 'Gender_Male_n', 'Gender_Male_% per Sector', 'Gender_Male_% per Social Category', 'Gender_Male_% per Workforce', 'Gender', 'Age_Older_n', 'Age_Older_% per Sector', 'Age_Older_% per Social Category', 'Age_Older_% per Workforce', 'Age_Younger_n', 'Age_Younger_% per Sector', 'Age_Younger_% per Social Category', 'Age_Younger_% per Workforce', 'Age', 'Sector_n', '% Sector per Workforce', 'Gender_Female', 'Gender_Male', 'Gender_Mixed', 'Age_Mixed', 'Age_Older', 'Age_Younger'], dt

In [74]:
df_jobs = categorize_df_gender_age(df_jobs)

In [75]:
df_jobs.columns


Index(['Search Keyword', 'Platform', 'Job ID', 'Job Title', 'Company Name', 'Location', 'Job Description', 'Rating', 'Employment Type', 'Company URL', 'Job URL', 'Job Age', 'Job Age Number', 'Collection Date', 'Data Row', 'Tracking ID', 'Industry', 'Job Date', 'Type of ownership', 'Language', 'Dutch Requirement in Job Ad', 'English Requirement in Job Ad', 'Sector Code', 'Sector', 'Keywords Count', 'Gender_Female_n', 'Gender_Female_% per Sector', 'Gender_Female_% per Social Category', 'Gender_Female_% per Workforce', 'Gender_Male_n', 'Gender_Male_% per Sector', 'Gender_Male_% per Social Category', 'Gender_Male_% per Workforce', 'Gender', 'Age_Older_n', 'Age_Older_% per Sector', 'Age_Older_% per Social Category', 'Age_Older_% per Workforce', 'Age_Younger_n', 'Age_Younger_% per Sector', 'Age_Younger_% per Social Category', 'Age_Younger_% per Workforce', 'Age', 'Sector_n', '% Sector per Workforce', 'Gender_Female', 'Gender_Male', 'Gender_Mixed', 'Age_Mixed', 'Age_Older', 'Age_Younger', 'Ge

In [76]:
get_df_info(df_jobs, ivs_all=ivs_all)


DF INFO:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17593 entries, 0 to 17592
Data columns (total 53 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Search Keyword                       17593 non-null  object  
 1   Platform                             17593 non-null  object  
 2   Job ID                               17593 non-null  object  
 3   Job Title                            17593 non-null  object  
 4   Company Name                         17591 non-null  object  
 5   Location                             17593 non-null  object  
 6   Job Description                      17593 non-null  object  
 7   Rating                               3780 non-null   float64 
 8   Employment Type                      17011 non-null  object  
 9   Company URL                          15954 non-null  object  
 10  Job URL                              17593 non-null  object  
 11  Job 

In [77]:
get_df_info(df_jobs, ivs_all=dvs)


DF INFO:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17593 entries, 0 to 17592
Data columns (total 53 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Search Keyword                       17593 non-null  object  
 1   Platform                             17593 non-null  object  
 2   Job ID                               17593 non-null  object  
 3   Job Title                            17593 non-null  object  
 4   Company Name                         17591 non-null  object  
 5   Location                             17593 non-null  object  
 6   Job Description                      17593 non-null  object  
 7   Rating                               3780 non-null   float64 
 8   Employment Type                      17011 non-null  object  
 9   Company URL                          15954 non-null  object  
 10  Job URL                              17593 non-null  object  
 11  Job 

In [78]:
df_jobs['Gender'].value_counts()


Male            6974
Mixed Gender    6664
Female          3955
Name: Gender, dtype: int64

In [79]:
df_jobs['Gender_Female'].value_counts()

0    13638
1     3955
Name: Gender_Female, dtype: int64

In [80]:
df_jobs['Gender_Num'].value_counts()

2    6974
1    6664
0    3955
Name: Gender_Num, dtype: int64

In [81]:
df_jobs['Platform'].value_counts()

LinkedIn     13813
Indeed        3198
Glassdoor      582
Name: Platform, dtype: int64

In [82]:
platform_order = ['LinkedIn', 'Indeed', 'Glassdoor']
df_jobs['Platform'] = df_jobs['Platform'].astype('category').cat.reorder_categories(platform_order, ordered=True)
df_jobs['Platform'] = pd.Categorical(df_jobs['Platform'], categories=platform_order, ordered=True)
df_jobs['Platform_Num'] = pd.to_numeric(df_jobs['Platform'].cat.codes).astype('int64')
df_jobs = df_jobs.join(pd.get_dummies(df_jobs[['Platform']], dtype='int64'))

In [83]:
# LinkedIn=13818, Indeed=3198, Glassdoor=3198
df_jobs[['Platform_Num', 'Platform_LinkedIn', 'Platform_Indeed', 'Platform_Glassdoor']].value_counts()

Platform_Num  Platform_LinkedIn  Platform_Indeed  Platform_Glassdoor
0             1                  0                0                     13813
1             0                  1                0                      3198
2             0                  0                1                       582
dtype: int64

In [84]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_including_sector_genage_data.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_including_sector_genage_data.csv', index=False)


# Use spacy to split job ads to sentences


### START HERE IF SOURCING FROM DF_JOBS_INCLUDING_SECTOR_GENAGE_DATA
### PLEASE SET CORRECT DIRECTORY PATHS BELOW


In [85]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [86]:
from setup_module.imports import *  # type:ignore # isort:skip # fmt:skip # noqa # nopep8


In [87]:
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_including_sector_genage_data.pkl')


In [88]:
df_jobs.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 17593 entries, 0 to 17592
Data columns (total 57 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Search Keyword                       17593 non-null  object  
 1   Platform                             17593 non-null  category
 2   Job ID                               17593 non-null  object  
 3   Job Title                            17593 non-null  object  
 4   Company Name                         17591 non-null  object  
 5   Location                             17593 non-null  object  
 6   Job Description                      17593 non-null  object  
 7   Rating                               3780 non-null   float64 
 8   Employment Type                      17011 non-null  object  
 9   Company URL                          15954 non-null  object  
 10  Job URL                              17593 non-null  object  
 11  Job Age        

In [89]:
df_jobs['Job Description'] = df_jobs['Job Description'].apply(lambda x: unicodedata.normalize('NFKD', x.encode('ascii', 'ignore').decode('utf-8', 'ignore')))


In [90]:
df_jobs['Job Description'].head()


0    About Our Client\nThe Global KYC organisation ...
1    Your role:\n\nWere in business to save our hom...
2    During the past four years Colourful Rebel has...
3    Job Description\nWe are currently recruiting a...
4    KARL LAGERFELD COMPANY PROFILE\n\nThe house of...
Name: Job Description, dtype: object

In [91]:
%%time
# Add sentencizer to spacy pipe and set custom punctuations
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_chars = pickle.load(f)

if 'sentencizer' not in nlp.pipe_names:
    sentencizer = nlp.add_pipe('sentencizer')
sentencizer.punct_chars.update(custom_punct_chars)

if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    with open(f'{data_dir}punctuations.txt', 'wb') as f:
        pickle.dump(sentencizer.punct_chars, f)

with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_chars = pickle.load(f)


CPU times: user 8.4 ms, sys: 4.66 ms, total: 13.1 ms
Wall time: 13.6 ms


In [92]:
# Add special cases to spacy
special_cases_dict = {
    'incl.': [{65: 'incl', 67: 'including'}],
    'incl. ': [{65: 'incl', 67: 'including'}],
    '(incl.': [{65: 'incl', 67: 'including'}],
    'etc.': [{65: 'etc', 67: 'et cetera'}],
    'etc. ': [{65: 'etc', 67: 'et cetera'}],
    'e.g.': [{65: 'e.g', 67: 'for example'}],
    'e.g. ': [{65: 'e.g', 67: 'for example'}],
    'andor ': [{65: 'andor', 67: 'and or'}],
    'and/or ': [{65: 'andor', 67: 'and or'}],
}

nlp.tokenizer.rules.update(special_cases_dict)


In [93]:
df_jobs['Job Description'].head()


0    About Our Client\nThe Global KYC organisation ...
1    Your role:\n\nWere in business to save our hom...
2    During the past four years Colourful Rebel has...
3    Job Description\nWe are currently recruiting a...
4    KARL LAGERFELD COMPANY PROFILE\n\nThe house of...
Name: Job Description, dtype: object

In [94]:
df_jobs['Job Description'].describe()


count                                                 17593
unique                                                17593
top       About Our Client\nThe Global KYC organisation ...
freq                                                      1
Name: Job Description, dtype: object

In [95]:
%%time
# Spacy sentencize
if all(custom_punct_char in sentencizer.punct_chars for custom_punct_char in custom_punct_chars):
    df_jobs['Job Description spacy_sentencized'] = df_jobs['Job Description'].progress_apply(
        lambda job_description: [
            sent
            for sentence in nlp(job_description).sents 
            for sent in re.split(pattern, sentence.text)
            if sent is not None
            and bool(sent) == True
            and len(sent) < 2
            and not isinstance(sent, str)
            and sent != '...'
            and sent not in nan_list
            and sent not in custom_punct_chars
            # and sent not in list(string.punctuation)
            # and sent not in list(string.whitespace)
            ]
    )

assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_sentencized.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_sentencized.csv', index=False)


progress-bar:   0%|          | 0/17593 [00:00<?, ?it/s]

CPU times: user 20min 3s, sys: 32.1 s, total: 20min 35s
Wall time: 25min 46s


In [96]:
# len=17593
len(df_jobs)


17593

In [97]:
df_jobs['Job Description spacy_sentencized'].head()


0    []
1    []
2    []
3    []
4    []
Name: Job Description spacy_sentencized, dtype: object

In [98]:
df_jobs['Job Description spacy_sentencized'].describe()


count     17593
unique        1
top          []
freq      17593
Name: Job Description spacy_sentencized, dtype: object

In [99]:
# Drop na, empty strings, punctuations, and keep only sentences of length more than 0
df_jobs = df_jobs.dropna(subset=['Job Description spacy_sentencized'])
df_jobs = df_jobs.loc[
    (df_jobs['Job Description spacy_sentencized'].isnull() == False)
    & (df_jobs['Job Description spacy_sentencized'].isna() == False)
    & (df_jobs['Job Description spacy_sentencized'].notnull() == True)
    & (df_jobs['Job Description spacy_sentencized'].notna() == True)
    & (df_jobs['Job Description spacy_sentencized'].astype(bool) == True)
    & (df_jobs['Job Description spacy_sentencized'].astype(str).apply(len) > 0)
    & (pd.isna(df_jobs['Job Description spacy_sentencized']) == False)
    & (df_jobs['Job Description spacy_sentencized'].astype(str).isin(custom_punct_chars) == False)
    & (df_jobs['Job Description spacy_sentencized'].astype(str).isin(non_whitespace_nan_list) == False)
    & (df_jobs['Job Description spacy_sentencized'].astype(str).isin(list(string.punctuation)) == False)
    & (df_jobs['Job Description spacy_sentencized'].astype(str).isin(list(string.whitespace)) == False)
]
df_jobs = df_jobs.dropna(subset=['Job Description spacy_sentencized'])


In [100]:
# len=17593
len(df_jobs)

0

In [101]:
df_jobs['Job Description spacy_sentencized'].head()


Series([], Name: Job Description spacy_sentencized, dtype: object)

In [102]:
# count=, unique=
df_jobs['Job Description spacy_sentencized'].describe()


count       0
unique      0
top       NaN
freq      NaN
Name: Job Description spacy_sentencized, dtype: object

In [103]:
# sentences=934
df_jobs['Job Description spacy_sentencized'].apply(len).max()


nan

In [104]:
df_jobs['Job Description spacy_sentencized'].loc[df_jobs['Job Description spacy_sentencized'].apply(len).idxmin()]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_986/3907542433.py:1 in <module>       │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_986/3907542433.py'                   │
│                                                                                                  │
│ /opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/pandas/core │
│ /series.py:2495 in idxmin                                                                        │
│                                                                                                  │
│   2492 │   │   >>> s.idxmin(skipna=False)                                                        │
│   2493 │   │   nan                                                                               │
│   2494 │   │   """                                                                               │
│ ❱ 2495 │   │   i = self.argmin(axis, skipna, *args, **kwargs)                                    │
│   2496 │   │   if i == -1:                                                                       │
│   2497 │   │   │   return np.nan                                                                 │
│   2498 │   │   return self.index[i]                                                              │
│                                                                                                  │
│ /opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/pandas/core │
│ /base.py:717 in argmin                                                                           │
│                                                                                                  │
│    714 │   │   else:                                                                             │
│    715 │   │   │   # error: Incompatible return value type (got "Union[int, ndarray]", expected  │
│    716 │   │   │   # "int")                                                                      │
│ ❱  717 │   │   │   return nanops.nanargmin(  # type: ignore[return-value]                        │
│    718 │   │   │   │   delegate, skipna=skipna                                                   │
│    719 │   │   │   )                                                                             │
│    720                                                                                           │
│                                                                                                  │
│ /opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/pandas/core │
│ /nanops.py:88 in _f                                                                              │
│                                                                                                  │
│     85 │   │   │   obj_iter = itertools.chain(args, kwargs.values())                             │
│     86 │   │   │   if any(self.check(obj) for obj in obj_iter):                                  │
│     87 │   │   │   │   f_name = f.__name__.replace("nan", "")                                    │
│ ❱   88 │   │   │   │   raise TypeError(                                                          │
│     89 │   │   │   │   │   f"reduction operation '{f_name}' not allowed for this dtype"          │
│     90 │   │   │   │   )                                                                         │
│     91 │   │   │   try:                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: reduction operation 'argmin' not allowed for this 

In [105]:
# Remove sentences with only punctuations, whitespace, and nan, etc from list of sentences
with open(f'{data_dir}punctuations.txt', 'rb') as f:
    custom_punct_chars = pickle.load(f)

df_jobs['Job Description spacy_sentencized'] = df_jobs['Job Description spacy_sentencized'].progress_apply(
    lambda job_sentences_list: [
        sentence.strip()
        for sentence in job_sentences_list
        if sentence is not None
        and len(sentence) > 5
        and sentence not in nan_list
        and sentence not in custom_punct_chars
        and sentence not in list(string.punctuation)
        and sentence not in list(string.whitespace)
    ]
)


progress-bar: 0it [00:00, ?it/s]

In [106]:
df_jobs['Job Description spacy_sentencized'].head()


Series([], Name: Job Description spacy_sentencized, dtype: object)

In [107]:
df_jobs['Job Description spacy_sentencized'].describe()


count       0
unique      0
top       NaN
freq      NaN
Name: Job Description spacy_sentencized, dtype: object

In [108]:
# Drop na, empty strings, punctuations, and keep only sentences of length more than 0
df_jobs = df_jobs.dropna(subset=['Job Description spacy_sentencized'])
df_jobs = df_jobs.loc[
    (df_jobs['Job Description spacy_sentencized'].isnull() == False)
    & (df_jobs['Job Description spacy_sentencized'].isna() == False)
    & (df_jobs['Job Description spacy_sentencized'].notnull() == True)
    & (df_jobs['Job Description spacy_sentencized'].notna() == True)
    & (df_jobs['Job Description spacy_sentencized'].astype(bool) == True)
    & (df_jobs['Job Description spacy_sentencized'].astype(str).apply(len) > 0)
    & (pd.isna(df_jobs['Job Description spacy_sentencized']) == False)
    & (df_jobs['Job Description spacy_sentencized'].astype(str).isin(custom_punct_chars) == False)
    & (df_jobs['Job Description spacy_sentencized'].astype(str).isin(non_whitespace_nan_list) == False)
    & (df_jobs['Job Description spacy_sentencized'].astype(str).isin(list(string.punctuation)) == False)
    & (df_jobs['Job Description spacy_sentencized'].astype(str).isin(list(string.whitespace)) == False)
]
df_jobs = df_jobs.dropna(subset=['Job Description spacy_sentencized'])


In [109]:
# len=17592
len(df_jobs)

0

In [110]:
df_jobs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 58 columns):
 #   Column                               Non-Null Count  Dtype   
---  ------                               --------------  -----   
 0   Search Keyword                       0 non-null      object  
 1   Platform                             0 non-null      category
 2   Job ID                               0 non-null      object  
 3   Job Title                            0 non-null      object  
 4   Company Name                         0 non-null      object  
 5   Location                             0 non-null      object  
 6   Job Description                      0 non-null      object  
 7   Rating                               0 non-null      float64 
 8   Employment Type                      0 non-null      object  
 9   Company URL                          0 non-null      object  
 10  Job URL                              0 non-null      object  
 11  Job Age                        

In [111]:
df_jobs['Job Description spacy_sentencized'].head()


Series([], Name: Job Description spacy_sentencized, dtype: object)

In [112]:
# count=17598, unique=17546
df_jobs['Job Description spacy_sentencized'].describe()


count       0
unique      0
top       NaN
freq      NaN
Name: Job Description spacy_sentencized, dtype: object

In [113]:
assert len(df_jobs) > 0 and isinstance(df_jobs, pd.DataFrame), f'ERORR: LENGTH OF DF = {len(df_jobs)}'
df_jobs.to_pickle(f'{df_save_dir}df_jobs_sentencized.pkl')
df_jobs.to_csv(f'{df_save_dir}df_jobs_sentencized.csv', index=False)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_986/3767229632.py:1 in <module>       │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_986/3767229632.py'                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AssertionError: ERORR: LENGTH OF DF = 0